In [1]:
# python
import sys
import importlib
# columnar analysis
from coffea.nanoevents import NanoAODSchema
from coffea import processor
# local
sidm_path = str(sys.path[0]).split("/sidm")[0]
if sidm_path not in sys.path: sys.path.insert(1, sidm_path)
from sidm.tools import sidm_processor, utilities
# always reload local modules to pick up changes during development
importlib.reload(sidm_processor)
importlib.reload(utilities)
# plotting
import matplotlib.pyplot as plt
utilities.set_plot_style()

In [2]:
samples = [
    '2Mu2E_500GeV_5p0GeV_8p0mm',
    'TTJets',
]

In [3]:
for sample in samples:
    print(sample, utilities.get_xs(sample))

Signal not in xs cfg, assuming 1fb
2Mu2E_500GeV_5p0GeV_8p0mm 0.001
TTJets 471.7


In [4]:
fileset = utilities.make_fileset([samples[0]], "llpNanoAOD_v2", max_files=2, location_cfg="signal_2mu2e_v10.yaml")
fileset = utilities.make_fileset([samples[1]], "llpNanoAOD_v2", max_files=2, location_cfg="backgrounds.yaml", fileset=fileset)

runner = processor.Runner(
    executor=processor.IterativeExecutor(),
    schema=NanoAODSchema,
    #maxchunks=1,
    skipbadfiles=True,
)

channels = [
    "2mu2e",
]
p = sidm_processor.SidmProcessor(
    channels,
    ["muon_base"],
)

output = runner.run(fileset, treename='Events', processor_instance=p)
out = output["out"]

Output()

Output()

/opt/conda/lib/python3.9/site-packages/awkward/_connect/_numpy.py:195: RuntimeWarning: invalid value encountered in
divide
  result = getattr(ufunc, method)(

/opt/conda/lib/python3.9/site-packages/awkward/_connect/_numpy.py:195: RuntimeWarning: divide by zero encountered 
in divide
  result = getattr(ufunc, method)(

#--------------------------------------------------------------------------
#                         FastJet release 3.4.0
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the GNU GPL v2 or higher.  
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code,
# CGAL and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------


In [5]:
for sample in samples:
    print(sample, out[sample]["metadata"])

2Mu2E_500GeV_5p0GeV_8p0mm {'nEvts': 4364}
TTJets {'nEvts': 46211}


In [9]:
print(samples[0])
out[samples[0]]["cutflow"][channels[0]].print_table()
print()
out[samples[0]]["cutflow"][channels[0]].print_table(unweighted=True)
print()
out[samples[0]]["cutflow"][channels[0]].print_table(fraction=True)
print()

2Mu2E_500GeV_5p0GeV_8p0mm
Signal not in xs cfg, assuming 1fb
cut name         individual cut N    all cut N
-------------  ------------------  -----------
No selection                 59.0         59.0
pass triggers                22.1         22.1
PV filter                    57.5         21.3
>=2 LJs                      11.4          4.9
2mu2e                        11.3          4.9

cut name         individual cut N    all cut N
-------------  ------------------  -----------
No selection               4364.0       4364.0
pass triggers              1633.0       1633.0
PV filter                  4253.0       1578.0
>=2 LJs                     843.0        361.0
2mu2e                       839.0        360.0

cut name         individual %    marginal %    cumulative %
-------------  --------------  ------------  --------------
No selection            100.0         100.0           100.0
pass triggers            37.4          37.4            37.4
PV filter                97.5          

In [10]:
print(samples[1])
out[samples[1]]["cutflow"][channels[0]].print_table()
print()
out[samples[1]]["cutflow"][channels[0]].print_table(unweighted=True)
print()
out[samples[1]]["cutflow"][channels[0]].print_table(fraction=True)
print()

TTJets
cut name         individual cut N    all cut N
-------------  ------------------  -----------
No selection           27830300.0   27830300.0
pass triggers            496851.3     496851.3
PV filter              27782722.7     495646.9
>=2 LJs                  158992.4       4215.7
2mu2e                     14453.9       4215.7

cut name         individual cut N    all cut N
-------------  ------------------  -----------
No selection              46211.0      46211.0
pass triggers               825.0        825.0
PV filter                 46132.0        823.0
>=2 LJs                     264.0          7.0
2mu2e                        24.0          7.0

cut name         individual %    marginal %    cumulative %
-------------  --------------  ------------  --------------
No selection            100.0         100.0           100.0
pass triggers             1.8           1.8             1.8
PV filter                99.8          99.8             1.8
>=2 LJs                   0.6    

In [11]:
output["processed"]

{WorkItem(dataset='2Mu2E_500GeV_5p0GeV_8p0mm', filename='root://xcache//store/group/lpcmetx/SIDM/ULSignalSamples/2018_v10/BsTo2DpTo2Mu2e/CutDecayFalse_SIDM_BsTo2DpTo2Mu2e_MBs-500_MDp-5p0_ctau-8p0_v3/LLPnanoAODv2/CutDecayFalse_SIDM_BsTo2DpTo2Mu2e_MBs-500_MDp-5p0_ctau-8p0_v3_part-0.root', treename='Events', entrystart=0, entrystop=4364, fileuuid=b'\xac\xd8R\xc6\xbf\x88\x11\xef\x9b\xf4[\x15\xe6\x9b\xbe\xef', usermeta={}),
 WorkItem(dataset='TTJets', filename='root://xcache//store/group/lpcmetx/SIDM//Backgrounds/2018_v2/TTJets_TuneCP5/LLPnanoAODv2/TTJets_TuneCP5_part1_part-0.root', treename='Events', entrystart=0, entrystop=46211, fileuuid=b'j\x19\x1eT(\xeb\x11\xef\xb8\xe7`\x15\xe6\x9b\xbe\xef', usermeta={})}